# Imports and Settings 

In [1]:
import sys
sys.path.append("../src")

import cv2
import dlib
import numpy as np
import pickle as pkl
from os.path import join
from icao_dataset import ICAODataset
from data_structures import Eye

In [2]:
PATH_TO_ROOT = ".."
EYE_INDEXES = [36, 39, 42, 45]

FILE_SHAPE_PREDICTOR = "shape_predictor_68_face_landmarks.dat"
FILE_DATASET = join(PATH_TO_ROOT, "data/cropped_faces_eyes/dataset.pkl")

print(FILE_DATASET)

..\data/cropped_faces_eyes/dataset.pkl


In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(FILE_SHAPE_PREDICTOR)

# Load Dataset 

In [4]:
dataset = pkl.load(open(FILE_DATASET, "rb"))

In [5]:
for _, im, mrk in dataset:
    image = im.astype(np.uint8)
    rects = detector(image, 1)
    
    if len(rects) > 0:
        shape = predictor(image, rects[0])
        coordinates = np.array([(shape.part(i).x, shape.part(i).y) for i in EYE_INDEXES]).flatten()
    else:
        coordinates = np.zeros(8, dtype=np.int)
    
    mrk.right_eye = Eye(*coordinates[:4])
    mrk.left_eye = Eye(*coordinates[4:])

In [6]:
FILE_OUTPUT = FILE_DATASET.replace(".pkl", "_dlib.pkl")

pkl.dump(dataset, open(FILE_OUTPUT, "wb"))